In [2]:
import tensorflow as tf

- Tensorのshapeは、[batch_size, q_length, depth]になる

batch_size：データ数

q_length:queryののトークンの長さ（e.g 好き、な、動物、は = 4）

depth：Embeddingした次元数

In [4]:
class SimpleAttention(tf.keras.models.Model):
    '''
    Attentioonの説明をするための、Multi-headではない単純なAttention
    '''
    def __init__(self, depth: int, *args, **kwargs):
        '''
        コンストラクタ
        :param depth: 隠れそう及び出力の次元
        '''
        super().__init__(*args, **kwargs)
        self.depth = depth
        
        self.q_dense_layer = tf.keras.layers.Dense(depth, use_bias=False, name='q_dense_layer')
        self.k_dense_layer = tf.keras.layers.Dense(depth, use_bias=False, name='d_dense_layer')
        self.v_dense_layer = tf.keras.layers.Dense(depth, use_bias=False, name='v_dense_layer')
        
        self.output_dense_layer = tf.keras.layers.Dense(depth, use_bias=False, name='output_dense_layer')
    
    def call(self, input: tf.Tensor, memory: tf.Tensor, attention_mask: tf.Tensor) -> tf.Tensor:
        '''
        モデルの実行
        :param input: queryのテンソル
        :param memory: queryに情報を与えるmemoryのテンソル
        :param attention_mask: attention weight に適用される mask
        '''
        q = self.q_dense_layer(input)  # [batch_size, q_length, depth]
        k = self.k_dense_layer(memory)  # [batch_size, m_length, depth]
        v = self.v_dense_layer(memory)
        
        q *= depth ** -0.5
        
        #ここでqとkの内積を取ることで、queryとkeyの関連度のようなものを計算する
        logit = tf.matmul(q, k, transpose_b=True)  # [batch_size, q_length, k_length]
        logit += tf.to_float(attention_mask) * input.dtype.min
        
        # softmaxを取ることで正規化します
        attention_weight = tf.nn.softmax(logit, name='attention_weight')
        
        #重みに従ってvalueから情報を引いてくる
        attention_output = tf.matmul(attention_weight, v)
        return self.output_dense_layer(attention_output)
        
        

## Attention の使い方
1. Self Attention

- セルフアテンションは、Itsが何を表してるかとかの照応関係をしめす
- セルフアテンションは、エンコーダーでもデコーダーでも利用する

2. SourceTarget-Attention

- これは、inputとmemoryを別々のTensorで扱う。
- デコーダーで利用される。
- デコーダーは、時刻tでのトークンから、時刻t+1でのトークンを予測する

対話では、 End-to-End Memory Networkも使えそう

In [5]:
attention_layer = SimpleAttention(depth=128)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### 学習効率がうまくなる仕組み
- Scaled Dot-Production

softmaxに入る値が多いと、softmaxの勾配が0に近づいてしまう。

その原因となるlogitは、行列積なのでdepthの次元数が大きいと大きくなってしまう。

なので、depthの大きさに従って小さくなるようにする

- Mask

attetionのweightをゼロにするためのマスクをする

PADやDecoderのセルフアテンションにおける未来を無視できるようにする仕組み

## Multi-head Attention